# **Cell 1: Instalasi Library**
*   **Tujuan: Menginstal semua library yang diperlukan untuk scraping dan pengolahan data**
*   **pengambilan konten web (requests), parsing HTML (beautifulsoup4), dan parser XML (lxml)**


In [ ]:
!pip install newspaper3k
!pip install pandas
!pip install requests
!pip install beautifulsoup4
!pip install lxml[html_clean]

import pandas as pd
import numpy as np
from newspaper import Article
from newspaper import Config
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import time
import random

print("Semua library berhasil diimport!")

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 83.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 11.0 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=8028e2f808114863f748efdf8ff756f7b02abdd1d21ff782989ed3dccbfe0450
  Stored in directory: /root/.cache/pip/wheels/a5/91/9f/00d66475960891a64867914273fcaf78df6cb04d905b104a2a
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3341 sha256=158203b301e1a8eb4881794342866075d0197a157d6dd11008a76d42bda24c8b
  Stored in directory: /root/.cache/pip/wheels/9f/9f/fb/364871d7426d3cdd4d293dcf7e53d97f1

# **Cell 2: Konfigurasi User Agent**
**Tujuan: Mengatur user agent untuk menghindari pemblokiran saat scraping**

In [ ]:
# Mengatur user agent untuk mensimulasikan browser
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'

# Mengkonfigurasi newspaper3k dengan user agent dan timeout
config = Config()
config.browser_user_agent = user_agent
config.request_timeout = 10

# **Cell 3: Daftar Sumber Berita**
*   **Tujuan: Menyediakan daftar URL sumber berita Indonesia yang relevan dengan topik AI**
*   **Sumber mencakup berbagai media seperti Katadata, DailySocial, Tech in Asia, dll.**

In [ ]:
ai_news_urls = [
    # Katadata - Teknologi & AI
    "https://katadata.co.id/berita/",
    "https://katadata.co.id/tag/artificial-intelligence",
    "https://katadata.co.id/tag/ai",

    # DailySocial - Tech Startup
    "https://dailysocial.id/tag/artificial-intelligence",
    "https://dailysocial.id/tag/ai",
    "https://dailysocial.id/category/digital-business",

    # Tech in Asia Indonesia
    "https://id.techinasia.com/topic/artificial-intelligence",
    "https://id.techinasia.com/topic/kecerdasan-buatan",

    # Kompas Tekno
    "https://tekno.kompas.com/tag/artificial-intelligence",
    "https://tekno.kompas.com/tag/ai",
    "https://tekno.kompas.com/tag/kecerdasan-buatan",

    # Tempo Tekno
    "https://tekno.tempo.co/tag/artificial-intelligence",
    "https://tekno.tempo.co/tag/ai",

    # CNN Indonesia Teknologi
    "https://www.cnnindonesia.com/teknologi",
    "https://www.cnnindonesia.com/tag/ai",
    "https://www.cnnindonesia.com/tag/artificial-intelligence",

    # Kumparan Tech
    "https://kumparan.com/kumparantech/tag/ai-1",
    "https://kumparan.com/kumparantech/tag/artificial-intelligence-1",

    # Kontan Teknologi
    "https://tekno.kontan.co.id/tag/ai",
    "https://tekno.kontan.co.id/tag/artificial-intelligence",

    # IDN Times Teknologi
    "https://www.idntimes.com/tech/tag/artificial-intelligence",
    "https://www.idntimes.com/tech/tag/ai",

    # VIVA Teknologi
    "https://www.viva.co.id/tag/ai",
    "https://www.viva.co.id/tag/artificial-intelligence",
]

print(f"Total sumber berita: {len(ai_news_urls)}")

Total sumber berita: 24


# **Cell 4: Fungsi Ekstraksi URL Artikel**

**Tujuan: Mendefinisikan fungsi untuk mengekstrak URL artikel dari halaman sumber**

In [ ]:
def extract_article_urls_from_page(page_url, max_articles=20):
    """
    Mengambil URL artikel dari halaman berita.
    Args:
        page_url (str): URL halaman sumber berita
        max_articles (int): Jumlah maksimum artikel yang diambil per halaman
    Returns:
        list: Daftar URL artikel yang ditemukan
    """
    article_urls = []
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(page_url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Mencari semua link yang kemungkinan adalah artikel
        links = soup.find_all('a', href=True)

        for link in links:
            href = link['href']
            text = link.get_text().strip()

            # Filter URL yang kemungkinan adalah artikel
            if (any(keyword in href.lower() for keyword in ['/artikel/', '/berita/', '/read/', '/news/', 'artificial-intelligence', '/ai/', 'kecerdasan-buatan'])
                and len(text) > 20):  # Judul artikel biasanya panjang

                # Normalisasi URL
                if href.startswith('/'):
                    base_url = '/'.join(page_url.split('/')[:3])
                    full_url = base_url + href
                else:
                    full_url = href

                # Hindari duplikat dan URL non-berita
                if (full_url not in article_urls and
                    not any(domain in full_url for domain in ['facebook.com', 'twitter.com', 'instagram.com']) and
                    len(full_url) > 20):
                    article_urls.append(full_url)

                # Batasi jumlah artikel per halaman
                if len(article_urls) >= max_articles:
                    break

    except Exception as e:
        print(f"Error extracting from {page_url}: {str(e)}")

    return article_urls[:max_articles]

# **Cell 5: Fungsi Ekstraksi Data Artikel**

**Tujuan: Mendefinisikan fungsi untuk mengekstrak konten artikel menggunakan newspaper3k**

In [ ]:
def get_article_data(url, article_id):
    """
    Mengambil data artikel menggunakan newspaper3k.
    Args:
        url (str): URL artikel
        article_id (int): ID unik untuk artikel
    Returns:
        dict: Data artikel (id, tanggal, sumber, url, judul, konten, penulis, ringkasan)
    """
    try:
        article = Article(url, config=config)
        article.download()
        article.parse()

        # Skip jika konten terlalu pendek
        if len(article.text.strip()) < 200:
            return None

        # Format tanggal
        publish_date = article.publish_date
        if publish_date:
            if isinstance(publish_date, str):
                publish_date = datetime.now().date()  # Fallback
            else:
                publish_date = publish_date.date()
        else:
            publish_date = datetime.now().date()

        # Extract domain sebagai source
        source = url.split('/')[2] if '/' in url else url

        return {
            'id': article_id,
            'date': publish_date,
            'source': source,
            'url': url,
            'title': article.title,
            'content': article.text,
            'authors': ', '.join(article.authors) if article.authors else '',
            'summary': article.meta_description
        }

    except Exception as e:
        print(f"Error processing {url}: {str(e)}")
        return None

# **Cell 6: Fungsi Filter Artikel AI**

**Tujuan: Mendefinisikan fungsi untuk memeriksa apakah artikel relevan dengan topik AI**

In [ ]:
# Daftar kata kunci terkait AI untuk filtering
ai_keywords = [
    'artificial intelligence', 'ai', 'kecerdasan buatan',
    'machine learning', 'pembelajaran mesin', 'deep learning',
    'chatgpt', 'openai', 'ibm granite', 'watsonx',
    'neural network', 'jaringan saraf', 'robotika', 'robotics',
    'automation', 'otomasi', 'data science', 'ilmu data',
    'nlp', 'natural language processing', 'pemrosesan bahasa alami',
    'computer vision', 'penglihatan komputer'
]

def contains_ai_keywords(text):
    """
    Memeriksa apakah teks mengandung kata kunci terkait AI.
    Args:
        text (str): Teks untuk diperiksa
    Returns:
        bool: True jika mengandung kata kunci AI, False jika tidak
    """
    if not text:
        return False
    text_lower = text.lower()
    return any(keyword in text_lower for keyword in ai_keywords)

# **Cell 7: Proses Pengumpulan Artikel**

**Tujuan: Menjalankan proses pengumpulan artikel dari semua sumber**

In [ ]:
print("Memulai proses pengumpulan artikel...")

# Inisialisasi variabel untuk menyimpan data
all_articles_data = []
article_id_counter = 0
collected_urls = set()

# Loop melalui setiap sumber berita
for source_url in ai_news_urls:
    print(f"\nMengumpulkan dari: {source_url}")

    try:
        # Dapatkan URL artikel dari halaman sumber
        article_urls = extract_article_urls_from_page(source_url, max_articles=15)
        print(f"Found {len(article_urls)} potential articles")

        # Proses setiap URL artikel
        for url in article_urls:
            if url in collected_urls:
                continue

            print(f"Processing: {url[:80]}...")

            # Ekstrak data artikel
            article_data = get_article_data(url, article_id_counter)

            if article_data:
                # Periksa apakah artikel relevan dengan topik AI
                title_content = f"{article_data['title']} {article_data['content']}".lower()

                if contains_ai_keywords(title_content):
                    all_articles_data.append(article_data)
                    collected_urls.add(url)
                    article_id_counter += 1
                    print(f"✓ Artikel {article_id_counter} berhasil dikumpulkan")
                else:
                    print("✗ Artikel tidak relevan (tidak mengandung kata kunci AI)")
            else:
                print("✗ Gagal mengekstrak artikel")

            # Delay untuk menghindari pemblokiran
            time.sleep(random.uniform(1, 3))

    except Exception as e:
        print(f"Error processing source {source_url}: {str(e)}")
        continue

print(f"\nTotal artikel berhasil dikumpulkan: {len(all_articles_data)}")

Memulai proses pengumpulan artikel...

Mengumpulkan dari: https://katadata.co.id/berita/
Found 15 potential articles
Processing: https://katadata.co.id/berita/nasional/68dd391b00f26/petugas-telah-padamkan-keba...
✓ Artikel 1 berhasil dikumpulkan
Processing: https://katadata.co.id/berita/energi/68dd347535700/utamakan-daya-beli-masyarakat...
✓ Artikel 2 berhasil dikumpulkan
Processing: https://katadata.co.id/berita/nasional/68dd32cdb71e7/serapan-anggaran-mbg-baru-r...
✓ Artikel 3 berhasil dikumpulkan
Processing: https://katadata.co.id/berita/nasional/68dd32a8854ab/pln-buka-lowongan-kerja-bag...
✓ Artikel 4 berhasil dikumpulkan
Processing: https://katadata.co.id/berita/nasional/68dd279b26289/keluarga-mahfud-md-keracuna...
✓ Artikel 5 berhasil dikumpulkan
Processing: https://katadata.co.id/berita/nasional/68dd255065a9a/rusun-hunian-pekerja-proyek...
✓ Artikel 6 berhasil dikumpulkan
Processing: https://katadata.co.id/berita/nasional/68dd1afa24be7/dpr-usulkan-hapus-kata-grat...
✓ Artikel 7 b

# **Cell 8: Simpan dan Analisis Data**

**Tujuan: Menyimpan data ke CSV dan menampilkan statistik dasar**

In [ ]:
# Menyimpan data ke CSV dan menampilkan analisis awal
if all_articles_data:
    # Konversi ke DataFrame
    df = pd.DataFrame(all_articles_data)

    # Urutkan kolom
    df = df[['id', 'date', 'source', 'url', 'title', 'content', 'authors', 'summary']]

    # Simpan ke CSV
    output_file = 'articles_raw.csv'
    df.to_csv(output_file, index=False, encoding='utf-8')

    print(f"Data berhasil disimpan ke {output_file}")
    print(f"Shape: {df.shape}")

    # Tampilkan sampel data
    print("\nSample data:")
    print(df[['id', 'source', 'title']].head())

    # Statistik dasar
    print(f"\nStatistik:")
    print(f"- Total artikel: {len(df)}")
    print(f"- Sumber berita unik: {df['source'].nunique()}")
    print(f"- Rentang tanggal: {df['date'].min()} hingga {df['date'].max()}")

    # Periksa panjang konten
    df['content_length'] = df['content'].str.len()
    print(f"- Rata-rata panjang konten: {df['content_length'].mean():.0f} karakter")

else:
    print("Tidak ada artikel yang berhasil dikumpulkan!")

Data berhasil disimpan ke articles_raw.csv
Shape: (61, 8)

Sample data:
   id          source                                              title
0   0  katadata.co.id  Petugas Telah Padamkan Kebakaran Hunian Pekerj...
1   1  katadata.co.id  Utamakan Daya Beli Masyarakat, Tarif Listrik T...
2   2  katadata.co.id  Serapan Anggaran MBG Baru Rp 21 T, BGN Optimis...
3   3  katadata.co.id  PLN Buka Lowongan Kerja bagi Lulusan D3 sampai...
4   4  katadata.co.id  Keluarga Mahfud MD Keracunan MBG, Kepala BGN M...

Statistik:
- Total artikel: 61
- Sumber berita unik: 8
- Rentang tanggal: 2025-04-29 hingga 2025-10-01
- Rata-rata panjang konten: 3093 karakter


# **Cell 9: Validasi dan Pembersihan Data**

**Tujuan: Memvalidasi data dan menyimpan versi yang telah dibersihkan**

In [ ]:
# Validasi dan pembersihan data
if all_articles_data:
    print("\nValidasi Data:")
    print(f"- Judul yang kosong: {df['title'].isnull().sum()}")
    print(f"- Konten yang kosong: {df['content'].isnull().sum()}")
    print(f"- URL yang kosong: {df['url'].isnull().sum()}")

    # Hapus duplikat berdasarkan URL
    initial_count = len(df)
    df = df.drop_duplicates(subset=['url'])
    final_count = len(df)
    print(f"- Duplikat dihapus: {initial_count - final_count}")

    # Simpan versi yang telah dibersihkan
    df.to_csv('articles_cleaned.csv', index=False, encoding='utf-8')
    print(f"\nData cleaned disimpan: articles_cleaned.csv ({len(df)} artikel)")


Validasi Data:
- Judul yang kosong: 0
- Konten yang kosong: 0
- URL yang kosong: 0
- Duplikat dihapus: 0

Data cleaned disimpan: articles_cleaned.csv (61 artikel)


# **Cell 10: Backup Manual URL Collection**

Tujuan: Menyediakan fungsi cadangan untuk pengumpulan artikel secara manual
**bold text**


In [ ]:
# Fungsi cadangan untuk pengumpulan artikel secara manual
def manual_url_collection():
    """
    Mengumpulkan URL artikel AI secara manual jika scraping otomatis gagal.
    Returns:
        list: Daftar URL artikel manual
    """
    manual_urls = [
        # Tambahkan URL artikel AI Indonesia yang penting di sini
        # Contoh:
        # "https://katadata.co.id/berita/2024/01/15/ai-dan-masa-depan-indonesia",
        # "https://dailysocial.id/post/penggunaan-ai-di-startup-indonesia",
    ]

    return manual_urls

# **Cell 11: Upload ke Google Drive (Opsional)**

**Tujuan: Menyimpan file ke Google Drive (opsional, tetap dikomentari)**

In [ ]:
# Opsional: Menyimpan file ke Google Drive
"""
from google.colab import drive
drive.mount('/content/drive')

# Salin file ke Google Drive
import shutil
shutil.copy('articles_raw.csv', '/content/drive/MyDrive/mapping-ai-adoption-id/data/raw/articles.csv')
shutil.copy('articles_cleaned.csv', '/content/drive/MyDrive/mapping-ai-adoption-id/data/raw/articles_cleaned.csv')

print("File berhasil disalin ke Google Drive")
"""

print("\n🎉 Data collection selesai!")
print("Lanjutkan ke tahap preprocessing...")


🎉 Data collection selesai!
Lanjutkan ke tahap preprocessing...
